In [52]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/farhanfahim/Documents/Retail-Sales-Forecast-ML/data/raw/financials.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Segment                700 non-null    object
 1   Country                700 non-null    object
 2    Product               700 non-null    object
 3    Discount Band         700 non-null    object
 4    Units Sold            700 non-null    object
 5    Manufacturing Price   700 non-null    object
 6    Sale Price            700 non-null    object
 7    Gross Sales           700 non-null    object
 8    Discounts             700 non-null    object
 9     Sales                700 non-null    object
 10   COGS                  700 non-null    object
 11   Profit                700 non-null    object
 12  Date                   700 non-null    object
 13  Month Number           700 non-null    int64 
 14   Month Name            700 non-null    object
 15  Year                   

In [55]:
df.columns.tolist()

['Segment',
 'Country',
 ' Product ',
 ' Discount Band ',
 ' Units Sold ',
 ' Manufacturing Price ',
 ' Sale Price ',
 ' Gross Sales ',
 ' Discounts ',
 '  Sales ',
 ' COGS ',
 ' Profit ',
 'Date',
 'Month Number',
 ' Month Name ',
 'Year']

In [56]:
print(df[' Units Sold '].head())
print(df['  Sales '].head())

0     $1,618.50 
1     $1,321.00 
2     $2,178.00 
3       $888.00 
4     $2,470.00 
Name:  Units Sold , dtype: object
0     $32,370.00 
1     $26,420.00 
2     $32,670.00 
3     $13,320.00 
4     $37,050.00 
Name:   Sales , dtype: object


In [10]:
pd.to_numeric(df['  Sales '], errors='coerce').head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name:   Sales , dtype: float64

In [57]:
# cleaning step 1 (removing trailing spaces from ALL column names)
for col in df.columns:
    df.columns = df.columns.str.strip()

# cleaning step 2 (removing $ and , then converting to float)
df['Sales'] = df['Sales'].astype(str)
df['Sales'] = df['Sales'].str.replace('$', '', regex=False)
df['Sales'] = df['Sales'].str.replace(',', '', regex=False)
df['Sales'] = df['Sales'].astype(float)

print(df.columns)
print(df.Sales.head())

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts',
       'Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name',
       'Year'],
      dtype='object')
0    32370.0
1    26420.0
2    32670.0
3    13320.0
4    37050.0
Name: Sales, dtype: float64


In [58]:
# cleaning step 3 (removing $ and , then converting to float for all price columns)
def clean_currency_column(df, column_name):
    df[column_name] = df[column_name].astype(str)
    df[column_name] = df[column_name].str.replace('$', '', regex=False)
    df[column_name] = df[column_name].str.replace(',', '', regex=False)
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    
clean_currency_column(df, 'Units Sold')
clean_currency_column(df, 'Manufacturing Price')
clean_currency_column(df, 'Sale Price')
clean_currency_column(df, 'Gross Sales')
clean_currency_column(df, 'Discounts')
clean_currency_column(df, 'COGS')
clean_currency_column(df, 'Profit')

In [59]:
# Changing date column into correct format
print(df['Date'].head())
df['Date'] = pd.to_datetime(df['Date'])
df.info()

0    01/01/2014
1    01/01/2014
2    01/06/2014
3    01/06/2014
4    01/06/2014
Name: Date, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Segment              700 non-null    object        
 1   Country              700 non-null    object        
 2   Product              700 non-null    object        
 3   Discount Band        700 non-null    object        
 4   Units Sold           700 non-null    float64       
 5   Manufacturing Price  700 non-null    float64       
 6   Sale Price           700 non-null    float64       
 7   Gross Sales          700 non-null    float64       
 8   Discounts            647 non-null    float64       
 9   Sales                700 non-null    float64       
 10  COGS                 700 non-null    float64       
 11  Profit               637 non-null    float6

In [60]:
# Making Buisness Sense
# Checking if Sales = Gross Sales - Discounts
sales_check = (df['Sales'] == df['Gross Sales'] - df['Discounts']).all()
print(f"Sales calculation is correct: {sales_check}")

# Check if Profit = Sales - COGS
profit_check = (df['Profit'] == df['Sales'] - df['COGS']).all()
print(f"Profit calculation is correct: {profit_check}")

Sales calculation is correct: False
Profit calculation is correct: False


In [61]:
# Checking with tolerance for floating point errors
np.isclose(df['Sales'], df['Gross Sales'] - df['Discounts']).all()

np.False_

In [62]:
# Creating a comparison dataframe (to know more about the missing values)
comparison = df[['Gross Sales', 'Discounts', 'Sales', 'COGS', 'Profit']].head(10)
comparison['Calculated_Sales'] = comparison['Gross Sales'] - comparison['Discounts']
comparison['Sales_Difference'] = comparison['Sales'] - comparison['Calculated_Sales']
comparison['Calculated_Profit'] = comparison['Sales'] - comparison['COGS']
comparison['Profit_Difference'] = comparison['Profit'] - comparison['Calculated_Profit']

print(comparison)

   Gross Sales  Discounts     Sales      COGS    Profit  Calculated_Sales  \
0      32370.0        NaN   32370.0   16185.0   16185.0               NaN   
1      26420.0        NaN   26420.0   13210.0   13210.0               NaN   
2      32670.0        NaN   32670.0   21780.0   10890.0               NaN   
3      13320.0        NaN   13320.0    8880.0    4440.0               NaN   
4      37050.0        NaN   37050.0   24700.0   12350.0               NaN   
5     529550.0        NaN  529550.0  393380.0  136170.0               NaN   
6      13815.0        NaN   13815.0    9210.0    4605.0               NaN   
7      30216.0        NaN   30216.0    7554.0   22662.0               NaN   
8      37980.0        NaN   37980.0   18990.0   18990.0               NaN   
9      18540.0        NaN   18540.0    4635.0   13905.0               NaN   

   Sales_Difference  Calculated_Profit  Profit_Difference  
0               NaN            16185.0                0.0  
1               NaN            1

In [63]:
# Looking at rows with missing Discounts
print("Rows with missing Discounts:")
print(df[df['Discounts'].isnull()][['Segment', 'Product', 'Discount Band', 'Discounts', 'Sales']].head())

# Looking at rows with missing Profit
print("\nRows with missing Profit:")
print(df[df['Profit'].isnull()][['Sales', 'COGS', 'Profit']].head())

Rows with missing Discounts:
      Segment      Product Discount Band  Discounts    Sales
0  Government   Carretera          None         NaN  32370.0
1  Government   Carretera          None         NaN  26420.0
2   Midmarket   Carretera          None         NaN  32670.0
3   Midmarket   Carretera          None         NaN  13320.0
4   Midmarket   Carretera          None         NaN  37050.0

Rows with missing Profit:
        Sales      COGS  Profit
187  136560.0  136560.0     NaN
189   95400.0   95400.0     NaN
193  358560.0  358560.0     NaN
200  136560.0  136560.0     NaN
209  128880.0  128880.0     NaN


In [64]:
# Filling with 0 when Discount Band is "None"
df['Discounts'] = df['Discounts'].fillna(0)

# Calculating from Sales - COGS (they are break-even transactions)
df['Profit'] = df['Profit'].fillna(df['Sales'] - df['COGS'])

print(df.isnull().sum())

# Verifying business logic now works or not
print("Sales calculation check:", 
      np.isclose(df['Sales'], df['Gross Sales'] - df['Discounts']).all())
print("Profit calculation check:", 
      np.isclose(df['Profit'], df['Sales'] - df['COGS']).all())

Segment                0
Country                0
Product                0
Discount Band          0
Units Sold             0
Manufacturing Price    0
Sale Price             0
Gross Sales            0
Discounts              0
Sales                  0
COGS                   0
Profit                 0
Date                   0
Month Number           0
Month Name             0
Year                   0
dtype: int64
Sales calculation check: True
Profit calculation check: True


In [65]:
df.to_csv('../data/processed/processed.csv', index=False)
print("Cleaned data saved")

Cleaned data saved
